In [0]:
from pyspark.sql.functions import col, lag, desc, row_number, count, when, lit, coalesce
from pyspark.sql.window import Window
from delta.tables import DeltaTable
import sys

# --- Widgets ---
dbutils.widgets.text("table_name", "maestro_demo", "Id de la tabla a validar")
dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_names", "framework_dq", "Esquema de UC donde residen las tablas")

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_names")

EXECUTION_TABLE = f"{CATALOG}.{SCHEMA}.dq_execution_traceability"
VALIDATIONS_TABLE = f"{CATALOG}.{SCHEMA}.dq_validations_traceability"
EVIDENCES_TABLE = f"{CATALOG}.{SCHEMA}.dq_evidences"

def main():
    try:
        #Recuperar ejecuciones a analizar
        df_exec = (
            spark.table(EXECUTION_TABLE)
            .filter(col("status").isin("SUCCESS", "FAILED"))
            .orderBy(col("execution_timestamp").desc())
        )

        if df_exec.count() == 0:
            return "No existen ejecuciones"

        latest_execution_id = df_exec.first().execution_id

        window_exec = Window.orderBy(col("execution_timestamp"))
        df_exec_with_prev = df_exec.withColumn(
            "prev_execution_id", lag("execution_id").over(window_exec)
        )

        prev_execution_id = (
            df_exec_with_prev.filter(col("execution_id") == latest_execution_id)
            .select("prev_execution_id")
            .first()
            .prev_execution_id
        )

        delta_validations = DeltaTable.forName(spark, VALIDATIONS_TABLE)
        delta_evidences = DeltaTable.forName(spark, EVIDENCES_TABLE)

        if prev_execution_id is None:
            print("Primera ejecución: todas las evidencias se marcan como nuevas")

            df_current = (
                spark.table(EVIDENCES_TABLE)
                .filter(col("execution_id") == latest_execution_id)
                .withColumn("is_new_failure", lit(1))
            )

            df_merge = (
                df_current.groupBy("execution_id", "validation_id")
                .agg(count("*").alias("new_failures"))
                .withColumn("persistent_failures", lit(0))
                .withColumn("resolved_failures", lit(0))
            )

            # MERGE validaciones
            delta_validations.alias("target").merge(
                df_merge.alias("source"),
                (col("target.execution_id") == col("source.execution_id"))
                & (col("target.validation_id") == col("source.validation_id")),
            ).whenMatchedUpdate(
                set={
                    "new_failures": col("source.new_failures"),
                    "persistent_failures": col("source.persistent_failures"),
                    "resolved_failures": col("source.resolved_failures"),
                }
            ).execute()

            # MERGE evidencias
            delta_evidences.alias("target").merge(
                df_current.alias("source"),
                (col("target.execution_id") == col("source.execution_id"))
                & (col("target.validation_id") == col("source.validation_id"))
                & (col("target.table_pk") == col("source.table_pk")),
            ).whenMatchedUpdate(
                set={"is_new_failure": col("source.is_new_failure")}
            ).execute()

            return "Primera ejecución procesada correctamente."

        else:
            # ============================================================
            # 3. SEGUNDA O SIGUIENTES — Cargar evidencias previas y actuales
            # ============================================================

            df_current = (
                spark.table(EVIDENCES_TABLE)
                .filter(col("execution_id") == latest_execution_id)
                .select(
                    col("evidence_id"),
                    col("execution_id"),
                    col("validation_id"),
                    col("table_pk")
                )
                .alias("curr")
            )

            df_prev = (
                spark.table(EVIDENCES_TABLE)
                .filter(col("execution_id") == prev_execution_id)
                .select(
                    col("evidence_id"),
                    col("execution_id"),
                    col("validation_id"),
                    col("table_pk")
                )
                .withColumnRenamed("evidence_id", "prev_evidence_id")
                .withColumnRenamed("execution_id", "prev_execution_id")
                .withColumnRenamed("validation_id", "prev_validation_id")
                .withColumnRenamed("table_pk", "prev_table_pk")
                .alias("prev")
            )

            # ============================================================
            # 4. Calcular is_new_failure (actual - prev)
            # ============================================================

            df_evidences = df_current.join(
                df_prev,
                (col("curr.validation_id") == col("prev.prev_validation_id"))
                & (col("curr.table_pk") == col("prev.prev_table_pk")),
                "left_outer",
            ).withColumn(
                "is_new_failure",
                when(col("prev.prev_validation_id").isNull(), lit(1)).otherwise(lit(0)),
            ).select(
                col("curr.execution_id"),
                col("curr.validation_id"),
                col("curr.table_pk"),
                col("is_new_failure"),
            )

            # ============================================================
            # 5. Calcular resolved_failures (prev - current)
            # ============================================================

            df_resolved = df_prev.join(
                df_current,
                (col("prev.prev_validation_id") == col("curr.validation_id"))
                & (col("prev.prev_table_pk") == col("curr.table_pk")),
                "left_anti",
            ).groupBy(
                col("prev_execution_id"),
                col("prev_validation_id"),
            ).agg(count("*").alias("resolved_failures"))

            # ============================================================
            # 6. Calcular new + persistent
            # ============================================================

            df_failures = (
                df_evidences.groupBy("execution_id", "validation_id")
                .agg(
                    count("is_new_failure").alias("new_failures"),
                    (count("*") - count("is_new_failure")).alias("persistent_failures"),
                )
            )

            df_failures = df_failures.join(
                df_resolved.withColumnRenamed("prev_execution_id", "execution_id")
                .withColumnRenamed("prev_validation_id", "validation_id"),
                ["execution_id", "validation_id"],
                "left",
            ).fillna({"resolved_failures": 0})


            # ============================================================
            # 7. MERGE FINAL
            # ============================================================

            # Validaciones
            delta_validations.alias("target").merge(
                df_failures.alias("source"),
                (col("target.execution_id") == col("source.execution_id"))
                & (col("target.validation_id") == col("source.validation_id")),
            ).whenMatchedUpdate(
                set={
                    "new_failures": col("source.new_failures"),
                    "persistent_failures": col("source.persistent_failures"),
                    "resolved_failures": col("source.resolved_failures"),
                }
            ).execute()

            # Evidencias
            delta_evidences.alias("target").merge(
                df_evidences.alias("source"),
                (col("target.execution_id") == col("source.execution_id"))
                & (col("target.validation_id") == col("source.validation_id"))
                & (col("target.table_pk") == col("source.table_pk")),
            ).whenMatchedUpdate(set={"is_new_failure": col("source.is_new_failure")}).execute()

            return "Ejecución procesada correctamente."

    except Exception as e:
        raise e

if __name__ == "__main__":
    try:
        # Ejecutamos la función principal
        result = main()
    except Exception as e:
        # Salida explícita en caso de fallo
        dbutils.notebook.exit(f"Fallo en la persistencia de evidencias: {e}")
    else:
        # Salida explícita en caso de éxito
        dbutils.notebook.exit(f"Éxito: evidencias procesadas.")